# MONAI Deploy App SDKを使ったMedNIST分類器アプリのデプロイメント

このチュートリアルでは、MONAI Deploy App SDKを使って学習したモデルを、推論を行うローカルプログラム、同じことを行うワークフロージョブ、Dockerコンテナによるワークフロー実行として実行可能な成果物にパッケージングするプロセスをデモしています。

このチュートリアルでは、[こちらのMONAIチュートリアル](https://github.com/Project-MONAI/tutorials/blob/master/2d_classification/mednist_tutorial.ipynb)のようにMedNIST分類器を学習させ、推論アプリケーションを実装＆パッケージ化し、ローカルでアプリケーションを実行することになります。



## MONAI Coreを用いたMedNIST分類器モデルの学習

### 環境のセットアップ

In [ ]:
# Install necessary packages for MONAI Core
!python -c "import monai" || pip install -q "monai[pillow, tqdm]"
!python -c "import ignite" || pip install -q "monai[ignite]"
!python -c "import gdown" || pip install -q "monai[gdown]"

# Install MONAI Deploy App SDK package
!python -c "import monai.deploy" || pip install -q "monai-deploy-app-sdk"

### importsのセットアップ

In [ ]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import shutil
import tempfile
import glob
import PIL.Image
import torch
import numpy as np

from ignite.engine import Events

from monai.apps import download_and_extract
from monai.config import print_config
from monai.networks.nets import DenseNet121
from monai.engines import SupervisedTrainer
from monai.transforms import (
    AddChannel,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
    EnsureType,
)
from monai.utils import set_determinism

set_determinism(seed=0)

print_config()

MONAI version: 0.6.0
Numpy version: 1.19.5
Pytorch version: 1.9.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 0ad9e73639e30f4f1af5a1f4a45da9cb09930179

Optional dependencies:
Pytorch Ignite version: 0.4.5
Nibabel version: 3.2.1
scikit-image version: 0.17.2
Pillow version: 8.3.1
Tensorboard version: 2.6.0
gdown version: 3.13.0
TorchVision version: 0.10.0
ITK version: 5.2.0
tqdm version: 4.62.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.8.0
pandas version: 1.1.5
einops version: 0.3.2

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



### データセットのダウンロード

MedNISTのデータセットは、[TCIA](https://wiki.cancerimagingarchive.net/display/Public/Data+Usage+Policies+and+Restrictions)、RSNA Bone Age Challenge(https://www.rsna.org/education/ai-resources-and-training/ai-image-challenge/rsna-pediatric-bone-age-challenge-2017),  [the NIH Chest X-ray dataset](https://cloud.google.com/healthcare/docs/resources/public-datasets/nih-chest)の複数のセットから収集されたものです。

このデータセットは、[Dr. Bradley J. Erickson M.D., Ph.D.](https://www.mayo.edu/research/labs/radiology-informatics/overview) (Department of Radiology, Mayo Clinic)により、クリエイティブ・コモンズ CC BY-SA 4.0 ライセンスの下で提供されているものである。

MedNISTデータセットを使用する場合は、出典を明記してください。

In [ ]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

resource = "https://drive.google.com/uc?id=1QsnnkvZyJPcbRoV_ArW8SnE1OTuoVbKE"
md5 = "0bc7306e7427e00ad1c5526a6677552d"

compressed_file = os.path.join(root_dir, "MedNIST.tar.gz")
data_dir = os.path.join(root_dir, "MedNIST")
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, root_dir, md5)

/tmp/tmpgh08b1ks


Downloading...
From: https://drive.google.com/uc?id=1QsnnkvZyJPcbRoV_ArW8SnE1OTuoVbKE
To: /tmp/tmpthbz6o8r/MedNIST.tar.gz
61.8MB [00:05, 10.7MB/s]


Downloaded: /tmp/tmpgh08b1ks/MedNIST.tar.gz
Verified 'MedNIST.tar.gz', md5: 0bc7306e7427e00ad1c5526a6677552d.
Writing into directory: /tmp/tmpgh08b1ks.


In [ ]:
subdirs = sorted(glob.glob(f"{data_dir}/*/"))

class_names = [os.path.basename(sd[:-1]) for sd in subdirs]
image_files = [glob.glob(f"{sb}/*") for sb in subdirs]

image_files_list = sum(image_files, [])
image_class = sum(([i] * len(f) for i, f in enumerate(image_files)), [])
image_width, image_height = PIL.Image.open(image_files_list[0]).size

print(f"Label names: {class_names}")
print(f"Label counts: {list(map(len, image_files))}")
print(f"Total image count: {len(image_class)}")
print(f"Image dimensions: {image_width} x {image_height}")

Label names: ['AbdomenCT', 'BreastMRI', 'CXR', 'ChestCT', 'Hand', 'HeadCT']
Label counts: [10000, 8954, 10000, 10000, 10000, 10000]
Total image count: 58954
Image dimensions: 64 x 64


### セットアップと学習

ここでは、transformシーケンスを作成し、ネットワークを学習させる。検証やテストは、実際に機能することが分かっており、ここでは必要ないので省略する。


(train_transforms)=

In [ ]:
train_transforms = Compose(
    [
        LoadImage(image_only=True),
        AddChannel(),
        ScaleIntensity(),
        RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True),
        RandFlip(spatial_axis=0, prob=0.5),
        RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5),
        EnsureType(),
    ]
)

In [ ]:
class MedNISTDataset(torch.utils.data.Dataset):
    def __init__(self, image_files, labels, transforms):
        self.image_files = image_files
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        return self.transforms(self.image_files[index]), self.labels[index]


# データセットとローダーが1つであれば、検証やテストは必要ありません。
train_ds = MedNISTDataset(image_files_list, image_class, train_transforms)
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=300, shuffle=True, num_workers=10)

In [ ]:
device = torch.device("cuda:0")
net = DenseNet121(spatial_dims=2, in_channels=1, out_channels=len(class_names)).to(device)
loss_function = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(net.parameters(), 1e-5)
max_epochs = 5

In [ ]:
def _prepare_batch(batch, device, non_blocking):
    return tuple(b.to(device) for b in batch)


trainer = SupervisedTrainer(device, max_epochs, train_loader, net, opt, loss_function, prepare_batch=_prepare_batch)


@trainer.on(Events.EPOCH_COMPLETED)
def _print_loss(engine):
    print(f"Epoch {engine.state.epoch}/{engine.state.max_epochs} Loss: {engine.state.output[0]['loss']}")


trainer.run()

Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448272031/work/c10/core/TensorImpl.h:1156.)


Epoch 1/5 Loss: 0.1811893731355667
Epoch 2/5 Loss: 0.08026652783155441
Epoch 3/5 Loss: 0.05008228123188019
Epoch 4/5 Loss: 0.01724996417760849
Epoch 5/5 Loss: 0.029151903465390205


ネットワークは`classifier.zip`という名前のTorchscriptオブジェクトとしてここに保存されます。

In [ ]:
torch.jit.script(net).save("classifier.zip")

## MONAI Deploy App SDKによるアプリケーションの実装とパッケージング

Torchscriptのモデル(`classifier.zip`)をもとに、入力されたJpeg画像を処理し、予測（分類）結果をJSONファイル(`output.json`)として書き出すアプリケーションを実装します。


### プリケーションクラスでの演算子の作成と接続

学習時の事前変換として、以下の train transformsを使用しました。


```{code-block} python
---
lineno-start: 1
emphasize-lines: 3,4,5,9
caption: |
    Train transforms used in training
---
train_transforms = Compose(
    [
        LoadImage(image_only=True),
        AddChannel(),
        ScaleIntensity(),
        RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True),
        RandFlip(spatial_axis=0, prob=0.5),
        RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5),
        EnsureType(),
    ]
)
```

`RandRotate`, `RandFlip`, `RandZoom` 変換は学習時にのみ使用され、推論時には不要である。

推論アプリケーションでは、2つのオペレータを定義します。

1. `LoadPILOperator` - 入力パスからJPEG画像をロードし、ロードされた画像オブジェクトを次のオペレータに渡します。
    - このオペレータは、*train_transforms* の `LoadImage(image_only=True)` と同様の働きをしますが、扱う画像は1枚だけです。
    - **Input**: ファイルパス ([`DataPath`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DataPath.html))
    - **Output**: メモリ上の画像オブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))
2. `MedNISTClassifierOperator` -  与えられた画像をMONAIの`Compose`クラスで前変換し、Torchscriptのモデル（`classifier.zip`)に送り、予測結果をJSONファイル(`output.json`)に書き出す。
    - プリトランスフォームは、3つのトランスフォームで構成されています。 -- `AddChannel`, `ScaleIntensity`, `EnsureType`.
    - **Input**: メモリ上の画像オブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))
    - **Output**: 予測結果(`output.json`)が書き込まれるフォルダパス([`DataPath`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DataPath.html))

アプリケーションのワークフローは以下のようになる。

```{mermaid}
%%{init: {"theme": "base", "themeVariables": { "fontSize": "16px"}} }%%

classDiagram
    direction LR

    LoadPILOperator --|> MedNISTClassifierOperator : image...image


    class LoadPILOperator {
        <in>image : DISK
        image(out) IN_MEMORY
    }
    class MedNISTClassifierOperator {
        <in>image : IN_MEMORY
        output(out) DISK
    }
```


#### インポートのセットアップ

必要なクラスやデコレーターをインポートして、`MEDNIST_CLASSES` を定義しましょう。

In [ ]:
import monai.deploy.core as md  # 'md' stands for MONAI Deploy (or can use 'core' instead)
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)
from monai.transforms import AddChannel, Compose, EnsureType, ScaleIntensity

MEDNIST_CLASSES = ["AbdomenCT", "BreastMRI", "CXR", "ChestCT", "Hand", "HeadCT"]

#### Operatorクラスの作成


##### LoadPILOperator

In [ ]:
@md.input("image", DataPath, IOType.DISK)
@md.output("image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["pillow"])
class LoadPILOperator(Operator):
    """与えられた入力（DataPath）から画像を読み込み、出力（Image）にnumpy配列を設定します。"""

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import numpy as np
        from PIL import Image as PILImage

        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # 最初のファイルを取る

        image = PILImage.open(input_path)
        image = image.convert("L")  # グレースケール画像に変換する
        image_arr = np.asarray(image)

        output_image = Image(image_arr)  # numpyの配列でImage domainオブジェクトを生成します。
        op_output.set(output_image)

##### MedNISTClassifierOperator

In [ ]:
@md.input("image", Image, IOType.IN_MEMORY)
@md.output("output", DataPath, IOType.DISK)
@md.env(pip_packages=["monai"])
class MedNISTClassifierOperator(Operator):
    """与えられた画像を分類し、クラス名を返す。"""

    @property
    def transform(self):
        return Compose([AddChannel(), ScaleIntensity(), EnsureType()])

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import json

        import torch

        img = op_input.get().asnumpy()  # (64, 64), uint8
        image_tensor = self.transform(img)  # (1, 64, 64), torch.float64
        image_tensor = image_tensor[None].float()  # (1, 1, 64, 64), torch.float32

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        image_tensor = image_tensor.to(device)

        model = context.models.get()  # TorchScriptModel オブジェクトを取得します。

        with torch.no_grad():
            outputs = model(image_tensor)

        _, output_classes = outputs.max(dim=1)

        result = MEDNIST_CLASSES[output_classes[0]]  # クラス名を取得する
        print(result)

        # 出力（フォルダ）パスを取得し、存在しない場合はフォルダを作成する。
        output_folder = op_output.get().path
        output_folder.mkdir(parents=True, exist_ok=True)

        # 結果を "output.json "に書き込む
        output_path = output_folder / "output.json"
        with open(output_path, "w") as fp:
            json.dump(result, fp)

#### アプリケーションクラスの作成

アプリケーションクラスは以下のようなものです。

`Application` クラスを継承した `App` クラスを定義する。

`loadPILOperator` と `MedNISTClassifierOperator` は `App` の `compose()` メソッドの `self.add_flow()` で接続される。

In [ ]:
@md.resource(cpu=1, gpu=1, memory="1Gi")
class App(Application):
    """MedNIST分類器のアプリケーションクラス。"""

    def compose(self):
        load_pil_op = LoadPILOperator()
        classifier_op = MedNISTClassifierOperator()

        self.add_flow(load_pil_op, classifier_op)

### アプリをローカルで実行する

テスト用の入力ファイルのパスを探してみましょう。

In [ ]:
test_input_path = image_files[0][0]
print(f"Test input file path: {test_input_path}")

Test input file path: /tmp/tmpgh08b1ks/MedNIST/AbdomenCT/007000.jpeg


Jupyterノートブックでアプリを実行することができます。

In [ ]:
app = App()

In [ ]:
app.run(input=test_input_path, output="output", model="classifier.zip")

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 14835, Operator ID: dd5dee72-9764-458a-9719-dc89f3cd14ea)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MedNISTClassifierOperator
Executing operator MedNISTClassifierOperator (Process ID: 14835, Operator ID: 9b032f84-6a73-4f59-9c56-d04efed5bdb5)
AbdomenCT
Done performing execution of operator MedNISTClassifierOperator



In [ ]:
!cat output/output.json

"AbdomenCT"

Jupyter notebook内でアプリケーションを検証したら、上記のコードを連結して、アプリケーション全体をファイル(`mednist_classifier_monaideploy.py`)として書き出し、以下の行を追加します。

```python
if __name__ == "__main__":
    App(do_run=True)
```

上記の行は `python` インタープリタを使ってアプリケーションコードを実行するために必要なものです。

In [ ]:
%%writefile mednist_classifier_monaideploy.py

# Copyright 2021 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import monai.deploy.core as md  # 'md' stands for MONAI Deploy (or can use 'core' instead)
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)
from monai.transforms import AddChannel, Compose, EnsureType, ScaleIntensity

MEDNIST_CLASSES = ["AbdomenCT", "BreastMRI", "CXR", "ChestCT", "Hand", "HeadCT"]


@md.input("image", DataPath, IOType.DISK)
@md.output("image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["pillow"])
class LoadPILOperator(Operator):
    """与えられた入力（DataPath）から画像を読み込み、出力（Image）にnumpy配列を設定します。"""

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import numpy as np
        from PIL import Image as PILImage

        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # 最初のファイルを取る

        image = PILImage.open(input_path)
        image = image.convert("L")  # グレースケール画像に変換する
        image_arr = np.asarray(image)

        output_image = Image(image_arr)  # numpyの配列でImage domainオブジェクトを生成します。
        op_output.set(output_image)


@md.input("image", Image, IOType.IN_MEMORY)
@md.output("output", DataPath, IOType.DISK)
@md.env(pip_packages=["monai"])
class MedNISTClassifierOperator(Operator):
    """与えられた画像を分類し、クラス名を返す。"""

    @property
    def transform(self):
        return Compose([AddChannel(), ScaleIntensity(), EnsureType()])

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import json

        import torch

        img = op_input.get().asnumpy()  # (64, 64), uint8
        image_tensor = self.transform(img)  # (1, 64, 64), torch.float64
        image_tensor = image_tensor[None].float()  # (1, 1, 64, 64), torch.float32

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        image_tensor = image_tensor.to(device)

        model = context.models.get()  # TorchScriptModel オブジェクトを取得します。

        with torch.no_grad():
            outputs = model(image_tensor)

        _, output_classes = outputs.max(dim=1)

        result = MEDNIST_CLASSES[output_classes[0]]  # クラス名を取得する
        print(result)

        # 出力（フォルダ）パスを取得し、存在しない場合はフォルダを作成する。
        output_folder = op_output.get().path
        output_folder.mkdir(parents=True, exist_ok=True)

        # 結果を "output.json "に書き込む
        output_path = output_folder / "output.json"
        with open(output_path, "w") as fp:
            json.dump(result, fp)


@md.resource(cpu=1, gpu=1, memory="1Gi")
class App(Application):
    """MedNIST分類器のアプリケーションクラス。"""

    def compose(self):
        load_pil_op = LoadPILOperator()
        classifier_op = MedNISTClassifierOperator()

        self.add_flow(load_pil_op, classifier_op)


if __name__ == "__main__":
    App(do_run=True)

Writing mednist_classifier_monaideploy.py


今回は、コマンドラインでアプリを実行してみましょう。

In [ ]:
!python mednist_classifier_monaideploy.py -i {test_input_path} -o output -m classifier.zip

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 18193, Operator ID: de9a33aa-0abb-4e64-88af-90b27617ff63)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MedNISTClassifierOperator
Executing operator MedNISTClassifierOperator (Process ID: 18193, Operator ID: 73bfa497-459c-4ef3-998a-8d162be57687)
Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448272031/work/c10/core/TensorImpl.h:1156.)
AbdomenCT
Done performing execution of operator MedNISTClassifierOperator



上記のコマンドは、以下のコマンドラインと同じです。

In [ ]:
!monai-deploy exec mednist_classifier_monaideploy.py -i {test_input_path} -o output -m classifier.zip

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 18328, Operator ID: 70e92517-e6ad-4d0a-aaff-2141c672d587)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MedNISTClassifierOperator
Executing operator MedNISTClassifierOperator (Process ID: 18328, Operator ID: a9a7fc21-b180-4981-b775-ea8736e805a2)
Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448272031/work/c10/core/TensorImpl.h:1156.)
AbdomenCT
Done performing execution of operator MedNISTClassifierOperator



In [ ]:
!cat output/output.json

"AbdomenCT"

### アプリのパッケージ化

<a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/developing_with_sdk/packaging_app.html">MONAI Application Packager</a>でアプリをパッケージ化しよう。

In [ ]:
!monai-deploy package mednist_classifier_monaideploy.py --tag mednist_app:latest --model classifier.zip  # -l DEBUG

Building MONAI Application Package... Done
[2021-09-20 17:01:24,898] [INFO] (app_packager) - Successfully built mednist_app:latest


:::{note}
MONAIアプリケーションパッケージ（Dockerイメージ）のビルドには、時間がかかることがあります。進捗を確認したい場合は、`-l DEBUG`オプションを使用します。

:::

Docker イメージが作成されていることが確認できます。

In [ ]:
!docker image ls | grep mednist_app

mednist_app                                                             latest                                   8c78cc6e0966        3 seconds ago       15.3GB


### パッケージ化されたアプリをローカルで実行する

パッケージ化されたアプリは、<a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/developing_with_sdk/executing_packaged_app_locally.html">MONAI Application Runner</a>を使ってローカルに実行することができます。

In [ ]:
# テスト用入力ファイルを'input'フォルダにコピーします。
!mkdir -p input && rm -rf input/*
!cp {test_input_path} input/

# アプリを起動する
!monai-deploy run mednist_app:latest input output

Checking dependencies...
--> Verifying if "docker" is installed...

--> Verifying if "mednist_app:latest" is available...

Checking for MAP "mednist_app:latest" locally
"mednist_app:latest" found.

Reading MONAI App Package manifest...
 > export '/var/run/monai/export/' detected
--> Verifying if "nvidia-docker" is installed...

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 1, Operator ID: 7bb4824c-ebc7-4801-a0c3-1c5525b132cf)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MedNISTClassifierOperator
Executing operator MedNISTClassifierOperator (Process ID: 1, Operator ID: d27f4a05-e557-49c3-8adf-08f83a860d14)
AbdomenCT
Done performing execution of operator MedNISTClassifierOperator



In [ ]:
!cat output/output.json

"AbdomenCT"

**Note**: 演習が終了したら、以下のスクリプトを実行してください。

In [ ]:
# Remove data files which is in the temporary folder
if directory is None:
    shutil.rmtree(root_dir)